In [1]:
import uproot 
import numpy as np
import coffea
from coffea.nanoaod import NanoEvents
from coffea.nanoaod.methods.leptons import *
from coffea import hist
import matplotlib.pyplot as plt

In [2]:
fname="/uscms/home/ahall/nobackup/public/forTamas/iDM_nanoaod.root"
events = NanoEvents.from_file(fname,methods={"ElectronBPark":Electron})

In [4]:
#cuts on MET and Jet
MET_mask = events.MET.pt >120
Jet_mask = events.Jet.pt.max() > 80
MET_Jet_mask = MET_mask & Jet_mask

soft_eles = events[MET_Jet_mask].ElectronBPark

GenPart_mask = ((abs(events[MET_Jet_mask].GenPart.pdgId) == 11) 
               & (events[MET_Jet_mask].GenPart.parent.pdgId == 1000023)
               & (events[MET_Jet_mask].GenPart.status == 1) 
               & (events[MET_Jet_mask].GenPart.hasFlags(["fromHardProcess", "isFirstCopy"])))

#delta_r criteria for matched and unmatched electrons
eles = soft_eles.cross(events[MET_Jet_mask].GenPart[GenPart_mask], nested=True)
#note that for 1-5GeV range I used 0.005
matched_mask = ((eles.i0.delta_r(eles.i1)).min() < 0.025) & (soft_eles.isPF == False)
unmatched_mask = ((eles.i0.delta_r(eles.i1)).min() >= 0.025) & (soft_eles.isPF == False)
matched_eles = soft_eles[matched_mask]
unmatched_eles = soft_eles[unmatched_mask]

In [5]:
def cut_pt(electrons):
    return electrons[
        (electrons.pt<1)
    ]

In [7]:
print(len(cut_pt(matched_eles).flatten()))
print(len(matched_eles[matched_eles.pt<1].flatten()))

37
37
